# Export Polygraphs to Stata

## Sample application

In [ ]:
import os

import networkx as nx
import dgl

import numpy as np

import polygraphs as pg
from polygraphs import hyperparameters as hparams
from polygraphs import ops


# Create a PolyGraph configuration
params = hparams.PolyGraphHyperParameters()

# Initial beliefs are random uniform between 0 and 1
params.init.kind = 'uniform'
# Chance that action B is better than action A
params.epsilon = 0.001

params.network.kind = 'complete'
params.network.size = 4

# Enable logging; print progress every 100 steps
params.logging.enabled = True
params.logging.interval = 100

# Take snapshots
params.snapshots.enabled = True
params.snapshots.interval = 100

params.simulation.steps = 0
params.simulation.repeats = 1

# Store results in directory
params.simulation.results = "data/test"

# Remove directory, if exists
if os.path.isdir(params.simulation.results):
    os.rmdir(params.simulation.results)

# Set seed
params.seed = 123456789

pg.random(params.seed)
_ = pg.simulate(params, op=ops.BalaGoyalOp)

## Export to Pajek

In [ ]:
import h5py

import os
import math

import networkx as nx
import dgl

import numpy as np

import polygraphs as pg
from polygraphs import ops


def export(directory, id):
    """
    Post-process graph snapshots
    """
    # Resulting hashtable
    ht = {}
    graphs, _ = dgl.load_graphs(os.path.join(directory, f"{id}.bin"))
    graph = graphs[0]
    # Remove self-loops and convert graph to networkx format
    graph = dgl.remove_self_loop(graph)
    graphx = dgl.to_networkx(graph)

    # Process beliefs; every node has attributes:
    # {
    #   0: {
    #       "0001": "0.5"
    #       "0002": "0.3"
    #       "0003": "..."
    #   }
    #   1: {...},
    #   2: {...},
    # }
    nodes = list(graphx.nodes)
    attrs = {nid: dict() for nid in nodes}
    fp = h5py.File(os.path.join(directory, f"{id}.hd5"), "r")
    # print(fp)
    _keys = [int(key) for key in fp.keys()]
    _keys = sorted(_keys)
    digits = len(str(_keys[-1]))
    for key in _keys:
        # Get node beliefs at specific step
        beliefs =list(fp[str(key)])
        assert len(beliefs) == len(nodes), "Mismatch between number of nodes and beliefs"
        s = f"{{:0{digits}}}".format(key)
        for nid in nodes:
            attrs[nid][s] = f"{beliefs[nid]:5.3f}"

    print(attrs)
    nx.set_node_attributes(graphx, attrs)
    nx.write_pajek(graphx, os.path.join(directory, f"{id}.net"))


export("data/test", 1)